# Exploring documents with LDA-based topic modeling

Adapted from: https://stackabuse.com/python-for-nlp-topic-modeling/

## Load the docs

Document loading is detailed in `clustering.ipynb`

In [12]:
import os
import time
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()

DATADIR = '../data/DocumentCloud/subset'

# The visualization section below can be very slow. Not recommended for larger data sets, but tweaking some
# of these values might be helpful
N_TOPICS = 5
TERMS_PER_TOPIC = 30
NGRAM_RANGE = (1,3)
LAMBDA_STEP = 0.05

def documents(datadir=DATADIR):
    for fn in os.listdir(datadir):
        yield open(os.path.join(datadir, fn)).read()
docs = [doc for doc in documents()]

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english', ngram_range=NGRAM_RANGE)
doc_term_matrix = count_vect.fit_transform(docs)
doc_term_matrix

<380x16454 sparse matrix of type '<class 'numpy.int64'>'
	with 124359 stored elements in Compressed Sparse Row format>

In [14]:
doc_term_matrix.indices

array([10288,  7482,  3545, ...,  5781,  4989,  2498], dtype=int32)

In [15]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=N_TOPICS, random_state=42)
LDA.fit(doc_term_matrix)

for i,topic in enumerate(LDA.components_):
    print(f'Topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-TERMS_PER_TOPIC:]])
    print('\n')

Topic #0:
['budget', '000 00 contract', 'adjourn', 'ordinance authorizing', 'review', 'authorizing', 'finance audit', 'business', 'audit', 'items', '00 contract number', '00 contract', '2017', 'approval', 'chairman', '2018', 'ordinance', 'finance', '2019', 'board', '000 00', 'approved', '000', 'agenda', 'minutes', 'contract number', 'motion', 'number', 'contract', '00']


Topic #1:
['work', 'motion', '2016', 'rail', 'information', 'customer', 'elevator', 'stated', 'facilitator', 'stations', 'serpe', 'minutes', 'asked', '2019', 'fuller', 'service', 'chairman', 'new', 'people', 'members', 'vs', 'ada', '2018', 'station', '2017', 'complaints', 'bus', 'cta', 'mr', 'ms']


Topic #2:
['longhini assistant', 'offices', 'longhini', 'gregory longhini', 'assistant secretary', 'gregory', 'west', '567', 'assistant', 'west lake', '567 west lake', '567 west', 'time', 'second', 'notice', 'regular', 'business', 'general', 'chicago transit board', 'chairman', 'transit board', 'secretary', 'transit author

In [16]:
start = time.time()
viz = pyLDAvis.sklearn.prepare(LDA, doc_term_matrix, count_vect, R=TERMS_PER_TOPIC, lambda_step=LAMBDA_STEP, sort_topics=False)
print('Visualization prepared in: ', round(time.time() - start), 'seconds')
viz

Visualization prepared in:  178 seconds


/Users/scott/repos/NUKnightLab/Documenters.org/venv/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.104070 -0.120488       1        1  15.317770
1     -0.083638  0.257853       2        1  26.695544
2      0.307550 -0.022777       3        1  18.205891
3     -0.022117  0.063935       4        1  20.602795
4     -0.097726 -0.178523       5        1  19.178000, topic_info=      Category         Freq             Term        Total  loglift  logprob
5130   Default   692.000000  chicago transit   692.000000  30.0000  30.0000
4199   Default  1122.000000            board  1122.000000  29.0000  29.0000
15544  Default   906.000000          transit   906.000000  28.0000  28.0000
5047   Default  1096.000000          chicago  1096.000000  27.0000  27.0000
10846  Default   390.000000               ms   390.000000  26.0000  26.0000
...        ...          ...              ...          ...      ...      ...
3209    Topic5   142.129967          approve   344.789348   0.7652  -5.4393
14031   Topic5   120.601446         seconded   234.401719   0.9869  -5.6035
13176   Topic5   124.635553          regular   350.074741   0.6187  -5.5706
5799    Topic5   112.292593     construction   236.266522   0.9076  -5.6749
10748   Topic5   118.743073               mr   548.383799   0.1214  -5.6191

[347 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
0          1  0.354695         00
0          2  0.009173         00
0          3  0.105491         00
0          4  0.472417         00
0          5  0.059626         00
...      ...       ...        ...
16408      1  0.166219        yes
16408      2  0.100919        yes
16408      5  0.730178        yes
16411      1  0.189021  yes votes
16411      5  0.810091  yes votes

[588 rows x 3 columns], R=30, lambda_step=0.05, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])